### Import ETF ownership and Fund ownership

In [16]:
import pandas as pd

ownership_path = output_file = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final.csv"
df_ownership = pd.read_csv(ownership_path)

display(df_ownership)

,date,stock_RIC,index_member,stock_value_held,FUND_stock_value_held,market_cap,ETF_ownership,FUND_ownership
0,2010-01-01,0MW4EUR.xbo^K15,0,14.731695,9.666637,NaN,NaN,NaN
1,2010-02-01,0MW4EUR.xbo^K15,0,11.226477,9.598608,NaN,NaN,NaN
2,2010-03-01,0MW4EUR.xbo^K15,0,9.887296,8.259427,NaN,NaN,NaN
3,2010-04-01,0MW4EUR.xbo^K15,0,11.585353,9.845493,NaN,NaN,NaN
4,2010-05-01,0MW4EUR.xbo^K15,0,15.047999,9.688948,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
196238,2023-08-01,ZURN.S,1,5870.052977,3874.537377,6.559551e+10,0.089489,0.059067
196239,2023-09-01,ZURN.S,1,5816.278767,3834.866671,6.521676e+10,0.089184,0.058802
196240,2023-10-01,ZURN.S,1,5789.045875,3802.279416,6.373943e+10,0.090824,0.059653
196241,2023-11-01,ZURN.S,1,6022.326667,3973.467134,6.283013e+10,0.095851,0.063241


## stock price, gross profitability

In [21]:
##### import data
import pandas as pd
path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/m_stock_level_data.csv"
df_add = pd.read_csv(path)

###### lag df_add by one month
df_add['date'] = pd.to_datetime(df_add['date'])
df_add['date'] = df_add['date'] + pd.DateOffset(months=1)

##### change df_ownership also to date format
df_ownership["date"] = pd.to_datetime(df_ownership['date'])


result_df = pd.merge(df_ownership, df_add[['date', 'stock_RIC', 'price', "return1Mo", "gross_profit", "price_to_BV"]], on=['date', 'stock_RIC'], how='left')

display(result_df)
output_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
result_df.to_csv(output_path, index=False)

,date,stock_RIC,index_member,stock_value_held,FUND_stock_value_held,market_cap,ETF_ownership,FUND_ownership,price,return1Mo,gross_profit,price_to_BV
0,2010-01-01,0MW4EUR.xbo^K15,0,14.731695,9.666637,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02-01,0MW4EUR.xbo^K15,0,11.226477,9.598608,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03-01,0MW4EUR.xbo^K15,0,9.887296,8.259427,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04-01,0MW4EUR.xbo^K15,0,11.585353,9.845493,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-05-01,0MW4EUR.xbo^K15,0,15.047999,9.688948,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
196257,2023-08-01,ZURN.S,1,5870.052977,3874.537377,6.559551e+10,0.089489,0.059067,440.363548,-0.648448,NaN,2.722117
196258,2023-09-01,ZURN.S,1,5816.278767,3834.866671,6.521676e+10,0.089184,0.058802,435.965935,-1.364793,NaN,2.732482
196259,2023-10-01,ZURN.S,1,5789.045875,3802.279416,6.373943e+10,0.090824,0.059653,433.448678,-2.094610,NaN,2.722944
196260,2023-11-01,ZURN.S,1,6022.326667,3973.467134,6.283013e+10,0.095851,0.063241,435.510247,-1.259506,NaN,2.903088


## dependent variable: daily volatility for each month

In [22]:
df_dependent = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")
df_dependent['date'] = pd.to_datetime(df_dependent['date'], errors='coerce')
df_dependent.dropna(subset=['date'], inplace=True)
df_dependent.dropna(subset=['return1D'], inplace=True)
# Group by 'stock_RIC' and month, then calculate standard deviation and count of 'returns'
monthly_stats = df_dependent.groupby(['stock_RIC', pd.Grouper(key='date', freq='M')])['return1D'].agg(['std', 'count']).reset_index()
monthly_stats.rename(columns={'std': 'monthly_std_dev', 'count': 'count_returns'}, inplace=True)

# Create a year-month identifier for merging
df_dependent['year_month'] = df_dependent['date'].dt.to_period('M')
monthly_stats['year_month'] = monthly_stats['date'].dt.to_period('M')

# Merge the calculated std dev and count back onto the original DataFrame
df_dependent = pd.merge(df_dependent, monthly_stats, on=['stock_RIC', 'year_month'], how='left')

# Ensure columns are sorted before dropping duplicates
if 'date_x' in df_dependent.columns and 'stock_RIC' in df_dependent.columns:
    df_dependent.sort_values(by=['stock_RIC', 'date_x'], ascending=[True, False], inplace=True)
    df_dependent = df_dependent.drop_duplicates(subset=['stock_RIC', 'year_month'], keep='first')
else:
    print("Error: Check your column names and data integrity.")

# Drop the helper 'year_month' column if no longer needed
df_dependent.drop('year_month', axis=1, inplace=True)
df_dependent.drop('index', axis=1, inplace=True)

df_dependent['date_x'] = df_dependent['date_x'].dt.date
df_dependent['date_x'] = pd.to_datetime(df_dependent['date_x'])
df_dependent.rename(columns={'date_x': 'date'}, inplace=True)

# Display the updated DataFrame
display(df_dependent)

,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price,date_y,monthly_std_dev,count_returns
1185542,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.133779,-0.066756,938065.0,1.404420e+07,7.302660e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11-30,0.008801,2076
1165736,0MW4EUR.xbo^K15,2015-10-30,14.950000,-0.066845,-0.333333,-0.133601,4538663.0,6.787420e+07,7.291920e+09,2.235776e+09,NaN,14.650000,15.260000,2015-10-31,0.149910,22
1145989,0MW4EUR.xbo^K15,2015-09-30,14.970000,0.066845,-0.200000,0.335121,7431477.0,1.112457e+08,7.305110e+09,2.235776e+09,NaN,14.930000,14.980000,2015-09-30,0.201650,22
1127377,0MW4EUR.xbo^K15,2015-08-31,14.940000,0.134048,0.403226,-1.059603,3412482.0,5.096370e+07,7.290470e+09,2.235776e+09,NaN,14.870000,15.000000,2015-08-31,0.498490,21
1106891,0MW4EUR.xbo^K15,2015-07-31,15.100000,0.599600,-0.132275,-0.264201,1315684.0,1.978780e+07,7.367691e+09,2.235776e+09,NaN,14.660000,15.410000,2015-07-31,0.483467,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78981,ZURN.S,2010-05-31,166.645847,0.042248,3.001305,-1.415487,286368.0,4.778093e+07,2.468892e+10,NaN,1.053197,166.645847,166.786595,2010-05-31,2.251636,19
63612,ZURN.S,2010-04-30,167.705219,-1.678264,-3.417772,-5.938007,1179841.0,1.999244e+08,2.474560e+10,NaN,1.100166,167.705219,167.844857,2010-04-30,1.236923,20
48149,ZURN.S,2010-03-31,189.913011,-0.331858,1.235955,4.362934,1636985.0,3.115723e+08,2.802245e+10,NaN,1.265999,189.842751,189.913011,2010-03-31,0.622809,23
30553,ZURN.S,2010-02-26,177.091914,1.092896,1.688261,13.199301,876093.0,1.546427e+08,2.612145e+10,NaN,1.219970,177.023538,177.091914,2010-02-28,1.339416,20


## merge dependent variable to monthly_panel

In [23]:
# Assume df_dependent is defined earlier in your script
df_dependent_sub = df_dependent[["stock_RIC", "date", "monthly_std_dev", "count_returns"]].copy()

# Drop rows with NaN in the 'date' column to ensure date operations succeed
df_dependent_sub.dropna(subset=['date'], inplace=True)

# Shift the 'date' to the first day of the next month
df_dependent_sub['date'] = df_dependent_sub['date'] + pd.DateOffset(days=1)

output_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
df_monthly = pd.read_csv(output_path)
df_monthly["date"] = pd.to_datetime(df_monthly['date'])

# Merge DataFrames
merged_df = pd.merge(df_monthly, df_dependent_sub, on=['date', 'stock_RIC'], how='left')
display(merged_df)

# Save the merged DataFrame
merged_df.to_csv(output_path, index = False)

,date,stock_RIC,index_member,stock_value_held,FUND_stock_value_held,market_cap,ETF_ownership,FUND_ownership,price,return1Mo,gross_profit,price_to_BV,monthly_std_dev,count_returns
0,2010-01-01,0MW4EUR.xbo^K15,0,14.731695,9.666637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02-01,0MW4EUR.xbo^K15,0,11.226477,9.598608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03-01,0MW4EUR.xbo^K15,0,9.887296,8.259427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04-01,0MW4EUR.xbo^K15,0,11.585353,9.845493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-05-01,0MW4EUR.xbo^K15,0,15.047999,9.688948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196257,2023-08-01,ZURN.S,1,5870.052977,3874.537377,6.559551e+10,0.089489,0.059067,440.363548,-0.648448,NaN,2.722117,0.821159,21.0
196258,2023-09-01,ZURN.S,1,5816.278767,3834.866671,6.521676e+10,0.089184,0.058802,435.965935,-1.364793,NaN,2.732482,0.727499,22.0
196259,2023-10-01,ZURN.S,1,5789.045875,3802.279416,6.373943e+10,0.090824,0.059653,433.448678,-2.094610,NaN,2.722944,NaN,NaN
196260,2023-11-01,ZURN.S,1,6022.326667,3973.467134,6.283013e+10,0.095851,0.063241,435.510247,-1.259506,NaN,2.903088,0.910886,22.0
